# Quantum Generative Adversarial Network for QFT State Preparation

Welcome to this challenge notebook! In this lab you will build the **ansatz circuit** for a **Quantum Generative Adversarial Network (qGAN)** to prepare quantum Fourier transform (QFT) states. The exercise is inspired by recent work on variational quantum algorithms and, in particular, the use of qGANs to learn probability distributions using a hybrid classical–quantum approach. If you are new to qGANs, take a moment to review the structure of a generator–discriminator pair, either from your course notes or by consulting the paper by [Cerezo et al. (2021)](https://arxiv.org/pdf/2012.09265). This notebook breaks the problem down into a sequence of tasks so that you can implement the core components yourself.

---

**📈 Difficulty**: ★★☆☆☆ (2 / 5)

---

## Background reading

Before diving into the implementation, you should have a basic understanding of how a qGAN is constructed. The generator is a quantum circuit that produces a parametrized quantum state, while the discriminator is a classical neural network trained to distinguish samples from the generator versus real data. A variational ansatz for the generator must be expressive enough to represent the target distribution but simple enough to train.

For further reading, you may consult the article by [Cerezo et al. (2021)](https://arxiv.org/pdf/2012.09265), *Variational Quantum Algorithms*, which provides an overview of variational circuits and training strategies.

## Installation Requirements

Before running this notebook, make sure the following packages are installed in your environment:

In [ ]:
# UNCOMMENT TO INSTALL
'''
%pip install qiskit
%pip install qiskit-aer
%pip install classiq
%pip install torch
'''
pass # DELETE THIS LINE WHEN INSTALLING

## Task 1 – Set up your environment

In this first task you will import the packages required for constructing and training a qGAN. We will be using **Classiq** to compile our variational circuit, **Qiskit** to convert between circuit representations, and **PyTorch** to build the discriminator.
Execute the following cell to import the necessary modules. If any import fails, make sure the relevant packages are installed in your environment.

In [ ]:
# === Imports ===
from qGAN import qGAN

from classiq import *

import qiskit.qasm3
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Sampler

import torch
import torch.nn as nn

from typing import List, Set

## Task 2 – Implement a variational ansatz

The generator of a qGAN is a variational quantum circuit whose parameters are trained during adversarial learning. Your goal in this task is to construct an ansatz that:

- Applies single‑qubit rotations *RY* and *RZ* with trainable parameters.
- Uses controlled *RZ* (*CRZ*) gates to entangle pairs of qubits.
- Contains a unique entangling pattern (for example, entangle neighbouring qubits on odd and even layers).
  - Get creative with this! A better QFT state similarity translates to a stronger final deep model!

Complete the skeleton below by filling in the loops and parameter calls.

In [ ]:
# Number of qubits in the system
N = 6  # feel free to experiment with this value

# Number of repeated layers in the ansatz
reps = 3  # increase for greater expressivity at the cost of more parameters

@qfunc
def main(
    x: CArray[CReal, N],
    p_crz: CArray[CArray[CReal, N], N],
    p_rz: CArray[CReal, N],
    p_ry: CArray[CReal, N],
    q_out: Output[QArray[QBit, N]]
) -> None:
    """Apply the variational ansatz `reps` times with parameter mapping.

    Parameters
    ----------
    x : CArray[CReal, N]
        Classical input features.
    p_crz : CArray[CArray[CReal, N], N]
        Parameters for the controlled RZ gates.
    p_rz : CArray[CReal, N]
        Parameters for the single‑qubit RZ rotations.
    p_ry : CArray[CReal, N]
        Parameters for the single‑qubit RY rotations.
    q_out : Output[QArray[QBit, N]]
        The output quantum register.

    Returns
    -------
    None
    """
    # TODO: Allocate the quantum register
    
    
    # TODO: iterate over the number of repetitions
        # TODO: apply single‑qubit rotations to each qubit
        #         You may want to multiply the parameter by the input feature x[i]
        
        # TODO: apply entangling CRZ gates between pairs of qubits
        #         Tune the ranges of i, j to define your entangling pattern

    
    # End of variational circuit


## Task 3 – Compile the quantum program with Classiq

Classiq provides a high‑level compiler for turning `qfunc` definitions into executable quantum programs. In this task you will define a function `construct_func` that compiles your variational circuit. The `Preferences` object can be used to specify a target backend (for example, an IBMQ system or simulator). Create a model from your `main` function and then synthesize it into a `QPROG` object.

In [ ]:
def construct_func() -> QuantumCircuit:
    """Compile the `main` qfunc into a quantum circuit.

    Returns
    -------
    QuantumCircuit
        A Qiskit circuit representation of the compiled variational program.
    """
    # OPTIONAL: specify your preferred backend; leave blank to use the default local simulator
    prefs = 
    
    # TODO: create a model from the `main` qfunc
    
    
    # TODO: synthesize the model into a QPROG
    qprog = 
    
    # Convert the QPROG to a Qiskit circuit via its QASM description
    qasm_str = qprog.qasm
    qc = qiskit.qasm3.loads(qasm_str)
    
    return qprog, qc

## Task 4 – Build the discriminator and prepare data

The discriminator in a qGAN distinguishes between real samples and those produced by the generator. In this task you will define a simple feed‑forward neural network using PyTorch and generate a batch of real data samples. Be sure that the dimensionality of your data matches the number of qubits in your generator (each qubit corresponds to two basis states, so a system of `N` qubits outputs vectors of length `2**N`). Optionally, you can instantiate a `Sampler` from Qiskit Aer for circuit simulation.

In [ ]:
# TODO: Define a simple discriminator network
disc_model = 

# Generate real data samples
# TODO: REPLACE with your actual training data if available
samples = torch.rand(1000, 2 ** N)

# Optional sampler for simulation; leave as is to use the default
sampler = Sampler()

## Task 5 – Compile and train the qGAN

You are now ready to put everything together. Use your `construct_func` to compile the variational circuit, convert it to a Qiskit `QuantumCircuit`, and then instantiate the `OptimizedqGAN` class. Specify the hyperparameters for training (number of qubits, layers, epochs, learning rates, etc.) and then call `compile_model` followed by `train`. Experiment with different settings to see how they affect the quality of the generated state.

In [ ]:
# Compile the quantum program
qprog, qc = construct_func()

# Check out your quantum circuit in Classiq
show(qprog)

# Check out your quantum circuit in qiskit - UNCOMMENT TO SHOW
# qc.draw("mpl")

In [ ]:
# TODO: Test out different parameters!
g_lr = 2e-4
d_lr = 1e-4
epchs = 300
b_size = 32
nse = True # set to False to remove noise

# Instantiate the qGAN model
qgan = qGAN(
    model=disc_model,
    samples=samples,
    n_qubits=N,
    k_layers=reps,
    epochs=epchs,
    batch_size=b_size,
    gen_lr=g_lr,
    disc_lr=d_lr,
    device="cuda" if torch.cuda.is_available() else "cpu",
    sampler=sampler,
    use_noise=nse,
    circuit=qc
)

# Compile and train the qGAN
qgan.compile_model()
qgan.train()

# Congratulations!

You've successfully completed the **Quantum GAN for QFT State Preparation** challenge!

Throughout this notebook, you have:

- Explored the architecture of a hybrid quantum-classical Generative Adversarial Network (qGAN)
- Designed and implemented a variational quantum ansatz using the Classiq framework
- Defined a parameterized quantum circuit compatible with QFT-like state generation
- Constructed a classical discriminator and integrated real vs. generated samples
- Trained a qGAN model using realistic data and visualized its performance
- Compiled and exported quantum circuits for execution on IBM Quantum backends

This project demonstrates your ability to bridge classical ML techniques with quantum computing tools. If you've reached this point, you now have hands-on experience building hybrid quantum models and optimizing circuits for a meaningful task. Well done!

---


## 🚧 What's Next?

Looking to take this further? Here are two great directions:

- **Model other quantum states**: Try GHZ, W, Bell, Cluster, or Haar-random states instead of QFT.
- **Experiment with new discriminators**: Use CNNs, transformers, or variational objectives to learn distributions differently.

Quantum ML is evolving fast — feel free to explore creatively!

---

## ✍️ Author

**Zachary Scott-Murphy**  
Researcher - Quantum Computing + Artificial Intelligence  
[LinkedIn](https://linkedin.com/in/zscottmurphy) • [GitHub](https://github.com/ZacSM)

---

## ☕ Support

If this project helped you learn or build something cool, consider buying me a coffee!

- [PayPal](paypal.me/ZacJSM)
- [Venmo](venmo.com/zacsmms)

Thanks for helping me continue creating open-access quantum learning repos + tools.

---

## 📄 License

MIT License  
Created: August 2025